<h3>Import libraries:</h3>

In [21]:
from swift import Swift
import roboticstoolbox as rtb
from spatialmath import SE3
import matplotlib.pyplot as plt
from math import pi, degrees, radians
import numpy as np
from spatialmath.base import tr2rpy
from mpl_toolkits.mplot3d import Axes3D

# https://stackoverflow.com/questions/44116194/import-a-function-from-another-ipynb-file
from ipynb.fs.full.inverse_kinematics_dnb_robot import *

%matplotlib qt

<h3>Define the robot:</h3>

In [2]:
# Define the lenghts
l1 = 10.3
l2 = 16.525
l3 = 15.645
l4 = 8.8
l5 = 7.4

# Adjust the lenghts to plot the robot properly with the library (otherwise, the axes are too small)
# https://la.mathworks.com/matlabcentral/answers/485353-frames-not-appearing-using-robotics-system-toolbox
scaleFactor = 50

l1 /= scaleFactor
l2 /= scaleFactor
l3 /= scaleFactor
l4 /= scaleFactor
l5 /= scaleFactor

# Notes:
# 1. For a revolute joint the theta parameter of the link is ignored, and q used instead.
# 2. For a prismatic joint the d parameter of the link is ignored, and q used instead.
# 3. The joint offset parameter is added to q before computation of the transformation matrix.

# Robot's links
link1 = rtb.RevoluteDH(d=l1, a=0.0, alpha=pi/2, offset=-pi/2, qlim=[0,pi])
link2 = rtb.RevoluteDH(d=0.0, a=l2, alpha=0.0, offset=0.0, qlim=[0,pi])
link3 = rtb.RevoluteDH(d=0.0, a=l3, alpha=0.0, offset=-radians(152), qlim=[0,pi])
link4 = rtb.RevoluteDH(d=0.0, a=l4, alpha=0.0, offset=0.0, qlim=[0,pi])

# https://petercorke.github.io/robotics-toolbox-python/arm_dh.html#roboticstoolbox.robot.DHLink.PrismaticDH

# Create the robot
robot = rtb.DHRobot(
    links=[link1, link2, link3, link4],
    name="D&B Robot",
    tool = SE3.Ty(-l5)
    )

# Home angles
qHome = [radians(90), radians(90), radians(0), radians(62)]

# Set home position
robot.addconfiguration_attr('home',qHome)

print(robot)
print(f'Robot is in 1:{scaleFactor} scale.')

DHRobot: D&B Robot, 4 joints (RRRR), dynamics, standard DH parameters
┌────────────┬───────┬────────┬───────┬──────┬────────┐
│     θⱼ     │  dⱼ   │   aⱼ   │  ⍺ⱼ   │  q⁻  │   q⁺   │
├────────────┼───────┼────────┼───────┼──────┼────────┤
│  q1 - 90°  │ 0.206 │      0 │ 90.0° │ 0.0° │ 180.0° │
│  q2        │     0 │ 0.3305 │  0.0° │ 0.0° │ 180.0° │
│  q3 - 152° │     0 │ 0.3129 │  0.0° │ 0.0° │ 180.0° │
│  q4        │     0 │  0.176 │  0.0° │ 0.0° │ 180.0° │
└────────────┴───────┴────────┴───────┴──────┴────────┘

┌──────┬───────────────────────────────────────┐
│ tool │ t = 0, -0.15, 0; rpy/xyz = 0°, 0°, 0° │
└──────┴───────────────────────────────────────┘

┌──────┬──────┬──────┬─────┬──────┐
│ name │ q0   │ q1   │ q2  │ q3   │
├──────┼──────┼──────┼─────┼──────┤
│ home │  90° │  90° │  0° │  62° │
└──────┴──────┴──────┴─────┴──────┘

Robot is in 1:50 scale.


In [3]:
# Plot home position
fig1 = plt.figure(figsize=(8, 8))  # Adjust this for larger or smaller axes
robot.plot(q=robot.home, backend='pyplot', fig = fig1)
plt.show()

<h3>Kinematics tests:</h3>

In [4]:
# Compute the forward kinematics
fk1 = robot.fkine(qHome)

# Get the real coordinates
fk1Escaled = SE3.Rt(fk1.R, fk1.t * scaleFactor)
# This SE3.Rt method takes a rotation matrix R and a translation vector t and creates the desired SE(3) transformation

print(fk1Escaled)

   1         0         0         16.14     
   0         0        -1         0         
   0         1         0         5.611     
   0         0         0         1         



In [5]:
# Compute the inverse kinematics
ik1 = robot.ikine_LM(fk1, joint_limits = True)

for i in range(len(ik1.q)):
    print(f'q{i+1} = {round(degrees(ik1.q[i]),2)}°.')

q1 = 90.0°.
q2 = 90.0°.
q3 = 0.0°.
q4 = 62.0°.


In [6]:
# We'll use our own inverse kinematics function which gives us easier control of the end effector's orientation

pxHome = fk1.A[:,3][0]
pyHome = fk1.A[:,3][1]
pzHome = fk1.A[:,3][2]

# The orientation of the end effector is defined with respect to the base frame (only the pitch angle can be controlled)
pitchAngle = radians(0)

qList = inverse_kinematics(pxHome, pyHome, pzHome, pitchAngle, scaleFactor)

for index,q in enumerate(qList):
    print(f'q{index+1}: {q}°.')

q1: 90.0°.
q2: 90.0°.
q3: 0.0°.
q4: 62.0°.


In [7]:
# Traslation vector and pitch angle for frame1
pxFrame1 = 19/scaleFactor
pyFrame1 = -8.2/scaleFactor
pzFrame1 = 0.2/scaleFactor
pitchAngleFrame1 = radians(0)

# Its orientation needs to be determined; it's only known that the pitch angle, in the RPY convention, is equal to zero (parallel to the surface)
qFrame1 = inverse_kinematics(pxFrame1, pyFrame1, pzFrame1, pitchAngleFrame1, scaleFactor)
TFrame1 = robot.fkine([radians(i) for i in qFrame1])

print('Joint angles required to achieve the pose (orientation and translation) of frame1:')
print(qFrame1)

print('\nHomogeneous transformation matrix (HTM) of {frame1} with respect to the base frame {0}:')
print(TFrame1)

# Extract RPY angles (mobile/current frame, ZYX convention)
rpyFrame1 = tr2rpy(TFrame1.R, order='zyx', unit='deg')

print('Euler angles in the ZYX convention with respect to the mobile/current frame:')
print(f'- Roll: {round(rpyFrame1[0],2)}°.')
print(f'- Pitch: {round(rpyFrame1[1],2)}°.')
print(f'- Yaw: {round(rpyFrame1[2],2)}°.')

# mobile/current frame, ZYX convention = fixed frame, XYZ convention
# https://mecademic.com/insights/academic-tutorials/space-orientation-euler-angles/
# https://bdaiinstitute.github.io/spatialmath-python/func_3d.html#spatialmath.base.transforms3d.tr2rpy

Joint angles required to achieve the pose (orientation and translation) of frame1:
[66.66, 51.15, 16.45, 84.39]

Homogeneous transformation matrix (HTM) of {frame1} with respect to the base frame {0}:
   0.9182    0.0001603 -0.3962    0.38      
  -0.3962   -6.915e-05 -0.9182   -0.164     
  -0.0001745  1         0         0.003953  
   0         0         0         1         

Euler angles in the ZYX convention with respect to the mobile/current frame:
- Roll: 90.0°.
- Pitch: 0.01°.
- Yaw: -23.34°.


<h3>Trajectory test:</h3>

In [8]:
# https://petercorke.github.io/robotics-toolbox-python/arm_trajectory.html#roboticstoolbox.tools.trajectory.trapezoidal

# https://la.mathworks.com/help/robotics/ref/trapveltraj.html
# https://la.mathworks.com/help/robotics/ug/choose-trajectories-for-manipulator-paths.html
# https://la.mathworks.com/help/robotics/ug/design-a-trajectory-with-velocity-limits-using-a-trapezoidal-velocity-profile.html
# https://medium.com/mathworks/trajectory-planning-for-robot-manipulators-522404efb6f0

# https://www.youtube.com/watch?v=f3JVWqsDhbE
# https://youtu.be/Fd7wjZDoh7g?si=Hia8kw2mjwnycN0J

# Servomotor MG996R ----> Operating speed: 0.17 s/60º (4.8 V), 0.14 s/60º (6 V) 
# 0.14 s/60º (6 V) --------> 7,14 °/s (max speed)

# Set the time required to complete the trajectory
timeStepsTraj1 = 100
totalTime = 10 # Seconds
timeTraj1 = np.linspace(0, totalTime, timeStepsTraj1)

In [9]:
# Compute the trajectory using a trapezoidal velocity profile
# tg = rtb.tools.trapezoidal(pxHome*scaleFactor, pxFrame1*scaleFactor, timeTraj1)
# tg.plot()

# Generate the trajectory using a trapezoidal interpolator
q0Traj1 = [i*scaleFactor for i in [pxHome, pyHome, pzHome]]
q1Traj1 = [i*scaleFactor for i in [pxFrame1, pyFrame1, pzFrame1]]
Traj1 = rtb.tools.trajectory.mtraj(rtb.tools.trapezoidal, q0Traj1, q1Traj1, timeTraj1)

# Plot the position, velocity and acceleration data of the trajectory
Traj1.plot()

In [23]:
# Create a new figure for 3D plotting
fig2 = plt.figure()
ax = fig2.add_subplot(111, projection='3d')

# Plot the 3D trajectory in the workspace
ax.plot(Traj1.s[:,0], Traj1.s[:,1], Traj1.s[:,2], label='3D Line')

# Add labels and title
ax.set_xlabel('X Label')
ax.set_ylabel('Y Label')
ax.set_zlabel('Z Label')
ax.set_title('Task-space trajectory')

# Show the plot
plt.show()

# We got a line in the cartesian space using a trapezoidal interpolator

In [27]:
# Let's repeat the same process using a quintic polynomial interpolator

# Generate the trajectory using a quintic polynomial interpolator
Traj1v2 = rtb.tools.trajectory.mtraj(rtb.tools.quintic, q0Traj1, q1Traj1, timeTraj1)

# Plot the position, velocity and acceleration data of the trajectory
Traj1v2.plot()

In [26]:
# Create a new figure for 3D plotting
fig3 = plt.figure()
ax = fig3.add_subplot(111, projection='3d')

# Plot the 3D trajectory in the workspace
ax.plot(Traj1v2.s[:,0], Traj1v2.s[:,1], Traj1v2.s[:,2], label='3D Line')

# Add labels and title
ax.set_xlabel('X Label')
ax.set_ylabel('Y Label')
ax.set_zlabel('Z Label')
ax.set_title('Task-space trajectory')

# Show the plot
plt.show()

# We got a line in the cartesian space using a quintic polynomial interpolator

In [18]:
# Let's do it again with initial and final velocity different from zero (impractical in this case, it's just for testing)

# Set the velocities (they could be different for each axis)
dq0Traj1v3 = 0.5
dq1Traj1v3 = 1

Traj1v3x = rtb.tools.trajectory.quintic(q0Traj1[0], q1Traj1[0], timeTraj1, dq0Traj1v3, dq1Traj1v3)
Traj1v3y = rtb.tools.trajectory.quintic(q0Traj1[1], q1Traj1[1], timeTraj1, dq0Traj1v3, dq1Traj1v3)
Traj1v3z = rtb.tools.trajectory.quintic(q0Traj1[2], q1Traj1[2], timeTraj1, dq0Traj1v3, dq1Traj1v3)

# Create a new figure for 3D plotting
fig4 = plt.figure()
ax = fig4.add_subplot(111, projection='3d')

# Plot the 3D trajectory in the workspace
ax.plot(Traj1v3x.s, Traj1v3y.s, Traj1v3z.s, label='3D Line')

# Add labels and title
ax.set_xlabel('X Label')
ax.set_ylabel('Y Label')
ax.set_zlabel('Z Label')
ax.set_title('Task-space trajectory')

# Show the plot
plt.show()

# We don't get a line in the cartesian space using a polynomial interpolator if the initial and final velocities are different from zero
# This also happens between viapoints

In [19]:
# We've only interpolated the position of the robot, but it's also required an orientation interpolation
# Position and orientation interpolation are two different issues, therefore they're done separately

# https://petercorke.github.io/robotics-toolbox-python/arm_trajectory.html#roboticstoolbox.tools.trajectory.ctraj
# We'll handle position and orientation interpolation through the ctraj function

# Set the time required to complete the trajectory
timeStepsTraj2 = 100
totalTime2 = 10 # Seconds
timeTraj2 = np.linspace(0, totalTime, timeStepsTraj1)

# Compute the Cartesian trajectory between two poses (position + orientation)
Traj2 = rtb.tools.trajectory.ctraj(fk1, TFrame1, timeTraj2)

# Afterwards, compute the inverse kinematics for each pose along the trajectory
# These joint configurations can be fed into a control system to execute and control the movements of the robotic arm

<h3>Resolved-Rate Motion Control:</h3>